[FProfile.jl](https://github.com/cstjean/FProfile.jl) provides an alternative interface for [Julia's sampling profiler](https://docs.julialang.org/en/latest/manual/profile/). Please read the introduction of that document before proceeding if you've never used `@profile`.

# Profiling

You can build a profile by calling `@fprofile(code, delay=0.001, n_samples=1000000)`:

In [1]:
using FProfile, Calculus

pd = @fprofile second_derivative(sin, 1.0)

ProfileData(40 backtraces)

`@fprofile(N, ...)` is shorthand for `@fprofile(for _ in 1:N ... end)`:

In [2]:
pd = @fprofile 1000000 second_derivative(sin, 1.0)

ProfileData(920 backtraces)

Do not forget that Julia compiles code the first time a function is run; if you do not want to measure compilation time, execute your code once before profiling.

# Flat view

FProfile's `flat` report is a [dataframe](http://juliadata.github.io/DataFrames.jl/stable/man/getting_started/#Getting-Started-1). No particular knowledge of dataframes is necessary. I'll provide a few common operations below.

In [3]:
using DataFrames

df = flat(pd)
head(df, 5)   # show only the first 5 rows (the 5 rows with the highest count)

,count_pct,stackframe,specialization,method,file,function,module
1,99.89,"execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154","MethodInstance for execute_request(::ZMQ.Socket, ::IJulia.Msg)","execute_request(socket, msg) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl:115",/Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl,IJulia.execute_request,IJulia
2,99.89,anonymous at <missing>:?,MethodInstance for missing_info(),missing_info() in FProfile.MissingInfo at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:25,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.MissingInfo.missing_info,FProfile.MissingInfo
3,99.89,macro expansion at FProfile.jl:52 [inlined],MethodInstance for missing_info(),missing_info() in FProfile.MissingInfo at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:25,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.MissingInfo.missing_info,FProfile.MissingInfo
4,99.89,(::IJulia.##14#17)() at task.jl:335,MethodInstance for (::IJulia.##14#17)(),(::IJulia.##14#17)() in IJulia at task.jl:335,task.jl,IJulia.#14,IJulia
5,99.89,eventloop(::ZMQ.Socket) at eventloop.jl:8,MethodInstance for eventloop(::ZMQ.Socket),eventloop(socket) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl:2,/Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl,IJulia.eventloop,IJulia


The first column shows what fraction of backtraces (in %) go through the `method at file:line_number` in the `stackframe` column. It's the same quantity as in `Base.Profile.print()`, except for recursive calls: if `f(1)` calls `f(0)`, that's 2 counts in Base's report, but only 1 count in FProfile.

The other columns unpack the `stackframe`; they are useful for selecting subsets of the table. For instance, if I only care about the `derivative` function, I might use

```julia
    df[df[:function].===derivative, :]
```

It is common to focus optimization efforts on one or more modules at a time (... the ones you're developing). `flat(pd, MyModule)` filters out other modules and adds a useful column: `end_count_percent` measures how much `MyModule`-specific work is done on that line.

For instance, in the code below, while the `do_computation()` call takes a long time (it has a high `count_percent`), it merely calls another `Main` function, so it has a low `end_count_percent`. `sum_of_sin` has `end_count_percent = 100%` because while it calls `sum` and `sin`, those are defined in another module (`Base`), and counted as external to `Main`.

`flat(pd, (Module1, Module2, ...))` is also accepted.

In [4]:
@noinline do_computation(n) = sum_of_sin(n)
@noinline sum_of_sin(n) = sum(sin, 1:n)
pd2 = @fprofile do_computation(10000000)
flat(pd2, Main)

,count_pct,end_count_pct,stackframe,specialization,method,file,function,module
1,100.0,0.0,do_computation(::Int64) at In[4]:1,MethodInstance for do_computation(::Int64),do_computation(n) in Main at In[4]:1,In[4],do_computation,Main
2,100.0,100.0,sum_of_sin(::Int64) at In[4]:2,MethodInstance for sum_of_sin(::Int64),sum_of_sin(n) in Main at In[4]:2,In[4],sum_of_sin,Main


It pays to make sure that functions with a high `end_count_percent` are [well optimized](https://docs.julialang.org/en/latest/manual/performance-tips/).

Another way to reduce the level of detail is to aggregate by `:specialization, :method, :file, :function`, or `:module`.

In [5]:
df_by_method = flat(pd, combineby=:method)

,count_pct,method,file,function,module
1,99.89,missing_info() in FProfile.MissingInfo at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:25,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.MissingInfo.missing_info,FProfile.MissingInfo
2,99.89,"include_string(txt::String, fname::String) in Base at loading.jl:515",loading.jl,include_string,Base
3,99.89,"execute_request(socket, msg) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl:115",/Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl,IJulia.execute_request,IJulia
4,99.89,(::IJulia.##14#17)() in IJulia at task.jl:335,task.jl,IJulia.#14,IJulia
5,99.89,"include_string(mod::Module, code::String, fname::String) in Compat at /Users/cedric/.julia/v0.6/Compat/src/Compat.jl:464",/Users/cedric/.julia/v0.6/Compat/src/Compat.jl,include_string,Compat
6,99.89,eventloop(socket) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl:2,/Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl,IJulia.eventloop,IJulia
7,99.57,"second_derivative(f::Function, x::Number) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:71",/Users/cedric/.julia/v0.6/Calculus/src/derivative.jl,Calculus.second_derivative,Calculus
8,53.91,"derivative(f::Function, ftype::Symbol, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:2",/Users/cedric/.julia/v0.6/Calculus/src/derivative.jl,Calculus.derivative,Calculus
9,19.57,"finite_difference_hessian(f::Function, g::Function, x::Number, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:224",/Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl,Calculus.finite_difference_hessian,Calculus
10,14.67,"finite_difference(f::Function, x::T, dtype::Symbol) where T<:Number in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:48",/Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl,Calculus.finite_difference,Calculus


You can see the context (caller/called functions) around each of these rows by passing it to `tree`:

In [6]:
tree(pd, df_by_method, 9)   # show the context of the 9th row of `df_by_method`

 180 ...ulus/src/derivative.jl:71; second_derivative(::Function, ::F...
  175 .../finite_difference.jl:224; finite_difference_hessian(::Func...
   112 .../finite_difference.jl:55; finite_difference(::Calculus.##1...
   20  .../finite_difference.jl:27; finite_difference(::Calculus.##1...
   3   .../finite_difference.jl:0; finite_difference(::Calculus.##1...
  5   .../finite_difference.jl:0; finite_difference_hessian(::Func...


Other useful dataframe commands:

```julia
df[[:count_percent, :method]]   # select only those two columns
sort(df, cols=:end_count_percent, rev=true)  # sort by end_count_percent
showall(df)   # show the whole dataframe
```

See `?flat` for more options.

#### Comparing results

Pass two `ProfileData` objects to `flat` to compare them. The third column is simply the difference between the first two.

In [7]:
pd2 = @fprofile 1000000 second_derivative(sin, 1.0)
flat(pd, pd2, combineby=:function)

,count_pct_1,count_pct_2,count_pct_diff,function
1,19.57,17.39,-2.18,Calculus.finite_difference_hessian
2,53.91,54.42,0.51,Calculus.derivative
3,14.67,14.4,-0.27,Calculus.finite_difference
4,99.57,99.59,0.02,Calculus.second_derivative
5,99.89,99.9,0.01,IJulia.execute_request
6,99.89,99.9,0.01,include_string
7,99.89,99.9,0.01,IJulia.eventloop
8,99.89,99.9,0.01,IJulia.#14
9,99.89,99.9,0.01,FProfile.MissingInfo.missing_info


Of course, this is most useful when comparing different algorithms or commits (see [Revise.jl](https://github.com/timholy/Revise.jl)). The differences in the above table are  just noise.

# Tree view

FProfile's tree view looks the same as `Base.Profile.print(format=:tree)`. The numbers represent raw counts. (If some branches seem out of place, see [this issue](https://github.com/JuliaLang/julia/issues/9689))

In [8]:
tr = tree(pd)

 919 ./task.jl:335; (::IJulia.##14#17)()
  919 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
   919 ...rc/execute_request.jl:154; execute_request(::ZMQ.Socket, ::...
    919 ...Compat/src/Compat.jl:464; include_string(::Module, ::Stri...
     919 ./loading.jl:515; include_string(::String, ::String)
      919 ./<missing>:?; FProfile.MissingInfo.missing_info
       919 ...ile/src/FProfile.jl:52; FProfile.MissingInfo.missing_info
        919 ./profile.jl:23; FProfile.MissingInfo.missing_info
         917 ...ile/src/FProfile.jl:67; FProfile.MissingInfo.missing_info
          913 .../src/derivative.jl:71; second_derivative(::Function...
           490 ...src/derivative.jl:3; derivative(::Function, ::Sy...
           175 ...ite_difference.jl:224; finite_difference_hessian(:...
            112 ...ite_difference.jl:55; finite_difference(::Calculu...
             2  ...ite_difference.jl:0; finite_difference(::Base.#...
             32 ...ite_difference.jl:27; finite_difference(::Base.#..

Like `flat` reports, trees can be aggregated by `:specialization, :method, :file, :function`, or `:module`:

In [9]:
tree(pd, combineby=:module)

 919 IJulia
  919 Compat
   919 Base
    919 FProfile.MissingInfo
     916 Calculus
      55 FProfile.MissingInfo
 1   Calculus


If you're only interested in a particular module/file/method/function, you can pass it to `tree`, along with an optional _neighborhood range_.

In [10]:
tr_deriv = tree(pd, second_derivative, -1:1)    # -1:1 = show one level of callers and one level of called functions

 916 ...rofile/src/FProfile.jl:67; FProfile.MissingInfo.missing_info
  913 ...lus/src/derivative.jl:71; second_derivative(::Function, ::F...
   490 ...lus/src/derivative.jl:3; derivative(::Function, ::Symbol,...
   175 .../finite_difference.jl:224; finite_difference_hessian(::Func...
   5   .../finite_difference.jl:0; finite_difference_hessian(::Func...
   5   ...lus/src/derivative.jl:0; derivative(::Function, ::Symbol,...
  3   ...lus/src/derivative.jl:0; second_derivative(::Function, ::F...


Trees are an indexable and filterable datastructure. Use `get_specialization, get_method, get_file, get_function, get_module, is_C_call` and `is_inlined` in your `filter` predicate.

In [11]:
tr_deriv[1,1]

913 ...ulus/src/derivative.jl:71; second_derivative(::Function, ::Fl...
 490 ...lus/src/derivative.jl:3; derivative(::Function, ::Symbol, ...
 175 .../finite_difference.jl:224; finite_difference_hessian(::Funct...
 5   .../finite_difference.jl:0; finite_difference_hessian(::Funct...
 5   ...lus/src/derivative.jl:0; derivative(::Function, ::Symbol, ...


# Backtraces

(if you want to build your own analysis)

The raw profiler data is available either through `Base.Profile.retrieve()`, or through `pd.data` and `pd.lidict`. However, you might find `FProfile.backtraces(::ProfileData)` more immediately useful. 

In [12]:
count, trace = backtraces(pd)[1]  # get the first unique backtrace
@show count                       # the number of times that trace occurs in the raw data
trace

count = 1


11-element Array{StackFrame,1}:
 (::IJulia.##14#17)() at task.jl:335                                  
 eventloop(::ZMQ.Socket) at eventloop.jl:8                            
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154
 include_string(::Module, ::String, ::String) at Compat.jl:464        
 include_string(::String, ::String) at loading.jl:515                 
 anonymous at <missing>:?                                             
 macro expansion at FProfile.jl:52 [inlined]                          
 macro expansion at profile.jl:23 [inlined]                           
 macro expansion at FProfile.jl:67 [inlined]                          
 second_derivative(::Function, ::Float64) at derivative.jl:71         
 derivative(::Function, ::Symbol, ::Symbol) at derivative.jl:3        

Use the `get_method, get_file, ...` functions on `StackFrame` objects (see above). `tree(pd::ProfileData)` is defined as `tree(backtraces(pd))`, and similarly for `flat`, so you can modify the backtraces and get a tree/flat view of the results.